In [19]:
import pandas as pd
import numpy as np
from collections import Counter
import pprint

In [20]:
df = pd.DataFrame(data=pd.read_csv('weatherid3.csv'))


In [35]:
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2) for prob in probs])


In [36]:
def entropy_of_list(a_list):
    cnt = Counter(x for x in a_list)
    no = len(a_list)*1.0 
    probs = [x/no for x in cnt.values()]
    return entropy(probs)

print(entropy_of_list(df['Answer']))

0.9402859586706309


In [29]:
def info_gain(df, split_attr,target_attr,trace=0 ):
    
    df_split = df.groupby(split_attr)
    
    nobs= len(df.index)*1.0
    
    df_agg_ent = df_split.agg({target_attr:[entropy_of_list,lambda x: len(x)/nobs]})
    df_agg_ent.columns = ['Entropy','PropObservations']
    
    new_entropy = sum(df_agg_ent['Entropy']*df_agg_ent['PropObservations'])
    old_entropy = entropy_of_list(df[target_attr])
    
    return old_entropy - new_entropy


In [30]:
def id3(df,target_attr,attr_list ,defualt_class=None ):
    cnt =  Counter(x for x in df[target_attr])
    
    if len(cnt)==1:
        return next(iter(cnt))
    elif df.empty or (not attr_list):
        return defualt_class
    else:
        defualt_class = max(cnt.keys())
        
        
        gain_of_all = [info_gain(df , attr, target_attr) for attr in attr_list]
        index_of_max= gain_of_all.index(max(gain_of_all))
        best_attr = attr_list[index_of_max]
        tree = {best_attr : {}}
        
        remianing_attr = attr_list.copy()
        remianing_attr.remove(best_attr)
        
        for attr_value,subset in df.groupby(best_attr):
            tree[best_attr][attr_value]=id3(subset,target_attr,remianing_attr,defualt_class)
            
        return tree
        
        

In [31]:
attr_list = list(df.columns)
attr_list.remove('Answer')
pprint.pprint(id3(df,'Answer',attr_list))

{'Outlook': {'overcast': 'yes',
             'rain': {'Wind': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'Humidity': {'high': 'no', 'normal': 'yes'}}}}
